In [97]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [100]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'basline' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


In [101]:
df_imgs

,Condition,FOV,Path
0,Control,2,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
1,Control,3,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
2,Control,4,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
3,Control,5,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
4,Control,6,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
5,Control,7,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
6,Control,8,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
7,Treated,2,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
8,Treated,3,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
9,Treated,4,Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...


# Plot examples

Revision Figure on tratement 
Need to look at FOV 6 and 7 and compared control and treated

In [102]:
import napari 

# remove first and last consecutive zeros along dimension
def removeFirstAndLast(zeroArray):
    
    # remove first
    for ii, val in enumerate(zeroArray):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            lowest = ii # index of last of the first consecutive zeros
            
    # remove last
    for ii, val in enumerate(np.flip(zeroArray)):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            highest = len(zeroArray) - ii # index of first of the last consecutive zeros            
    
    return (lowest, highest)

# remove black borders from napari screenshot (2D image)
def removeBorders(RGB):
    rowMin = []
    rowMax = []
    colMin = []
    colMax = []
    for jj in range(3): # each RGB
        
        img = RGB[:, :, jj]
        # find first and last rows and cols of all zeros
        # remove any rows of zeros
        zeroRows = np.all(img == 0, axis = 1)
        zeroCols = np.all(img == 0, axis = 0)

        posRows = removeFirstAndLast(zeroRows)
        posCols = removeFirstAndLast(zeroCols)            
        
        rowMin.append(posRows[0])
        rowMax.append(posRows[1])
        colMin.append(posCols[0])
        colMax.append(posCols[1])
        
    # make sure border signal is not cut out
    rowMin = np.min(rowMin)
    rowMax = np.max(rowMax)
    colMin = np.min(colMin)
    colMax = np.max(colMax)
        
    cropped = RGB[rowMin: rowMax, colMin: colMax, :] 
    
    return cropped

In [103]:
# Read PPI 
PPI_save_path =  data_dir / 'basline' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)
df = df.rename(columns={'x': 'row', 'y': 'column'})
df.head()

,Cyto,row,column,PPI,Nuclei,Condition,FOV
0,0,2068,1445,CycD1/CDK4,0.0,Control,6
1,0,2068,1445,CycD1/CDK4,0.0,Control,6
2,0,2068,1445,CycD1/CDK4,0.0,Control,6
3,0,2068,1445,CycD1/CDK4,0.0,Control,6
4,0,724,3260,CycD1/CDK4,0.0,Control,6


In [104]:
image_dir = (Path().cwd().parents[0] / 'figures').absolute() / 'PLA_scatter'
image_dir.mkdir(parents=True, exist_ok=True)

## B06

In [105]:
# Get images
index = 4
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Control
FOV                                                          6
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
Name: 4, dtype: object


In [106]:
# Define matching bbox to figures
x_min, y_min = 1740, 1317
x_max, y_max = 2496, 1953
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_name = 'CycD1/CDK4'
df_ppi = df[(df.Condition == row.Condition) & (df.FOV == row.FOV)]
df_ppi = df_ppi[df_ppi.PPI == PPI_name]
df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
points = df_ppi[['row', 'column']].to_numpy()
points = points - np.array([y_min, x_min])

In [107]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points, size=8,face_color='red',
    face_colormap='red')

# Get screenshot
screenshot = viewer.screenshot(size=dapiImg.shape)
viewer.close()
# Save image
tiff.imsave(image_dir / 'B06.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_46900\1425072318.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / 'B06.tif', removeBorders(screenshot))


## E06

In [108]:
# Get images
index = 11 
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Treated
FOV                                                          6
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
Name: 11, dtype: object


In [109]:
# Define matching bbox to figures
x_min, y_min = 260,1736
x_max, y_max = 856,2284
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_name = 'CycD1/CDK4'
df_ppi = df[(df.Condition == row.Condition) & (df.FOV == row.FOV)]
df_ppi = df_ppi[df_ppi.PPI == PPI_name]
df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
points = df_ppi[['row', 'column']].to_numpy()
points = points - np.array([y_min, x_min])

In [110]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points, size=8,face_color='red',
    face_colormap='red')

# Get screenshot
screenshot = viewer.screenshot(size=dapiImg.shape)
viewer.close()
# Save image
tiff.imsave(image_dir / 'E06.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_46900\3723181325.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / 'E06.tif', removeBorders(screenshot))


## B07

In [111]:
# Get images
index = 5
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Control
FOV                                                          7
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
Name: 5, dtype: object


In [112]:
# Define matching bbox to figures
x_min, y_min = 1940,212
x_max, y_max = 2620,824
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_name = 'Sox2/Oct4'
df_ppi = df[(df.Condition == row.Condition) & (df.FOV == row.FOV)]
df_ppi = df_ppi[df_ppi.PPI == PPI_name]
df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
points = df_ppi[['row', 'column']].to_numpy()
points = points - np.array([y_min, x_min])

In [113]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points, size=8,face_color='red',
    face_colormap='red')

# Get screenshot
screenshot = viewer.screenshot(size=dapiImg.shape)
viewer.close()
# Save image
tiff.imsave(image_dir / 'B07.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_46900\298074377.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / 'B07.tif', removeBorders(screenshot))


## E07

In [114]:
# Get images
index = 12
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Treated
FOV                                                          7
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\basl...
Name: 12, dtype: object


In [115]:
# Define matching bbox to figures
x_min, y_min = 3320,2820
x_max, y_max = 3972,3400
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_name = 'Sox2/Oct4'
df_ppi = df[(df.Condition == row.Condition) & (df.FOV == row.FOV)]
df_ppi = df_ppi[df_ppi.PPI == PPI_name]
df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
points = df_ppi[['row', 'column']].to_numpy()
points = points - np.array([y_min, x_min])

In [116]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points, size=8,face_color='red',
    face_colormap='red')

# Get screenshot
screenshot = viewer.screenshot(size=dapiImg.shape)
viewer.close()
# Save image
tiff.imsave(image_dir / 'E07.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_46900\897212305.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / 'E07.tif', removeBorders(screenshot))
